In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import descartes
from shapely.geometry import Polygon
import geopandas as gpd
import os.path
from PIL import Image
import rasterio

## global variables
CLIP = False
INTERSEC = True

#set working directory
os.chdir("/Users/aminaly/Box Sync/mountain_biodiversity")



In [ ]:
## Read in all the files
kba = gpd.read_file(os.getcwd() + "/data/KBA/KBA2020/KBAsGlobal_2020_September_02_POL.shp")
wdpa = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/WDPA_Jun2021_Public_flattened.shp")
gmba = gpd.read_file(os.getcwd() + "/data/GMBA/Gmba_Inventory_GME_210420_Sel_292_attr/Gmba_Inventory_GME_210420_Sel_292_attr.shp")
    

In [14]:
 if(CLIP):
    
    #list of ISOs to use to clip kba & wdpa
    wrld_cntries = ['CHE', 'DEU', 'GHA', 'KOR', 'ZAF']
    
    #clip kba and wdpa using the list of isos 
    kba_c = kba[kba['ISO3'].isin(wrld_cntries)]
    wdpa = wdpa[wdpa['ISO3'].isin(wrld_cntries)]
    
    #gmba will be clipped a little differently. Doesn't have ISOs so we'll use a world shapefile
    world = gpd.read_file(os.getcwd() + "/data/World/world_shp/world.shp")
    world = world[world['CNTRY_NAME'].isin(kba_c['Country'].unique())] 
    gmba_c = gpd.overlay(gmba, world, how="intersection")
    #then we find a list of all the ranges included in the clip, and select those specifically from the main gmba
    gmba_c = gmba[gmba.GMBA_V2_ID.isin(gmba_c.GMBA_V2_ID)]
    
    #Once we've clipped them, save them out as shapefiles
    kba_c.to_file(os.getcwd() + "/data/KBA/KBA2020/clipped_KBAsGlobal_2020_September_02_POL.shp", 
                        driver='ESRI Shapefile')

    wdpa.to_file(os.getcwd() + 
                         "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/clipped_WDPA_Jun2021_Public_flattened.shp",
                       driver='ESRI Shapefile')

    gmba_c.to_file(os.getcwd() + "/data/GMBA/Gmba_Inventory_GME_210420_Sel_292_attr/clipped_Gmba_Inventory_GME_210420_Sel_292_attr.shp", 
                        driver='ESRI Shapefile')

In [2]:
## This section creates an intersection between the KBA and GMBA, splitting up the KBAs and only keeping
## the parts that overlap

if(CLIP):
    gmba_kba = gpd.overlay(kba_c, gmba_c, how = "intersection")
    gmba_kba.to_file(os.getcwd() + "/data/Combined/clipped_gmba_kba.shp", driver = 'ESRI Shapefile')
else: 
    gmba_kba = gpd.overlay(kba, gmba, how = "intersection")
    gmba_kba.to_file(os.getcwd() + "/data/Combined/gmba_kba.shp", driver = 'ESRI Shapefile')


In [6]:
## This section uses the above intersection to pull out all the KBAs that were included in full intersection
## and we create a new KBA layer that is a subset of the original KBA file with the whole KBA polygon
##(no clipping to mountain range)
#read in KBA
kba = gpd.read_file(os.getcwd() + "/data/KBA/KBA2020/KBAsGlobal_2020_September_02_POL.shp")

# read in clipped intersection and original
gmba_kba_clipped = gpd.read_file(os.getcwd() + "/data/Combined/clipped_gmba_kba.shp")
gmba_clipped = gpd.read_file(os.getcwd() + "/data/GMBA/Gmba_Inventory_GME_210420_Sel_292_attr/clipped_Gmba_Inventory_GME_210420_Sel_292_attr.shp")

#filter original kba by the SiteRecIds that show up & save
gmba_clipped_kba_full = kba[kba.SitRecID.isin(gmba_kba_clipped.SitRecID.unique())]

#select GMBA columns that we need for identifyers and do a spatial join based on intersections
gmba_kba

gmba_clipped_kba_full.to_file(os.getcwd() + "/data/Combined/clipped_full_kba_gmba_kba.shp", driver = 'ESRI Shapefile')

## Now do this for the non-clipped file
# read in unclipped intersection
gmba_kba = gpd.read_file(os.getcwd() + "/data/Combined/gmba_kba.shp")
gmba = gpd.read_file(os.getcwd() + "/data/GMBA/Gmba_Inventory_GME_210420_Sel_292_attr/Gmba_Inventory_GME_210420_Sel_292_attr.shp")


#filter original kba by the SiteRecIds that show up
gmba_kba_full = kba[kba.SitRecID.isin(gmba_kba.SitRecID.unique())]

# save out
gmba_kba_full.to_file(os.getcwd() + "/data/Combined/full_kba_gmba_kba.shp", driver = 'ESRI Shapefile')

In [3]:
gmba = gpd.read_file(os.getcwd() + "/data/GMBA/Gmba_Inventory_GME_210420_Sel_292_attr/Gmba_Inventory_GME_210420_Sel_292_attr.shp")
gmba_kba_full = gpd.read_file(os.getcwd() + "/data/Combined/full_kba_gmba_kba.shp")

gmba_s = gmba[['GMBA_V2_ID', 'RangeNameM', 'geometry']]



In [6]:
gmba_kba_full_a = gpd.sjoin(gmba_kba_full, gmba_s, how="inner", op='intersects')
gmba_kba_full_a.head()

,SitRecID,Country,ISO3,NatName,IntName,SitArea,IbaStatus,KbaStatus,AzeStatus,AddedDate,ChangeDate,Source,DelTxt,DelGeom,Shape_Leng,Shape_Area,geometry,index_right,GMBA_V2_ID,RangeNameM
0,19,Andorra,AND,IBA D'Andorra,Pirineo de Andorra,47463,confirmed,None,None,2009-06-17,None,IBA Directory description used to define bound...,Political layer used to provide boundary data,Draft polygon,1.060533,0.052022,"POLYGON ((1.43992 42.60649, 1.45041 42.60597, ...",62,11756,Pyrenees
3301,2161,France,FRA,Rochers de Biarritz : le Bouccalot et la Roche...,Rochers de Biarritz : le Bouccalot et la Roche...,128259,confirmed,None,None,2008-06-13,2014-07-16,LPO as part of Marine IBA update in July 2014,Marine buffers provided by LPO merged with exi...,Refined polygon,1.899874,0.143142,"POLYGON ((-1.56759 43.48252, -1.56942 43.48164...",62,11756,Pyrenees
3302,2162,France,FRA,"Haute Soule : Massif Forestier, Gorges d'Holza...","Haute Soule : Massif Forestier, Gorges d'Holza...",2333,confirmed,None,None,2008-06-13,None,Boundaries were originally digitized in 1993 f...,None,Refined polygon,0.335632,0.002575,"POLYGON ((-0.89868 42.99779, -0.89633 42.99655...",62,11756,Pyrenees
3303,2163,France,FRA,Haute Soule : ForÃªt des Arbailles,Haute Soule : ForÃªt des Arbailles,6895,confirmed,None,None,2008-06-13,None,Boundaries were originally digitized in 1993 f...,None,Refined polygon,0.629421,0.007628,"POLYGON ((-1.06136 43.17318, -1.05877 43.17311...",62,11756,Pyrenees
3304,2164,France,FRA,"Haute Soule : ForÃªt d'Irraty, Organbidexka et...","Haute Soule : ForÃªt d'Irraty, Organbidexka et...",5262,confirmed,None,None,2008-06-13,None,Boundaries were originally digitized in 1993 f...,None,Refined polygon,0.531523,0.005812,"POLYGON ((-1.11407 43.02543, -1.11375 43.02588...",62,11756,Pyrenees


In [7]:
gmba_kba_full_a.to_file(os.getcwd() + "/data/Combined/full_kba_gmba_kba.shp", driver = 'ESRI Shapefile')

wrld_cntries = ['CHE', 'DEU', 'GHA', 'KOR', 'ZAF']
gmba_kba_full_a_clipped = gmba_kba_full_a[gmba_kba_full_a['ISO3'].isin(wrld_cntries)]

gmba_kba_full_a_clipped.to_file(os.getcwd() + "/data/Combined/clipped_full_kba_gmba_kba.shp", driver = 'ESRI Shapefile')
